# Strategy

There are quite a few tools for generating PDFs. The most popular is [reportlab](https://pypi.python.org/pypi/reportlab) (people seem to recommend reportlab's [platypus](http://www.reportlab.com/apis/reportlab/2.4/platypus.html) for "simple" pdf [generation](https://www.reportlab.com/docs/platypus-example.py)).

I would love to do something like produce a [Markdown](http://daringfireball.net/projects/markdown/syntax) document (see [CommonMark](http://commonmark.org/)) and convert it to a PDF, but I can't find a pure-Python toolchain, and I don't know what you'll have installed. Similarly, I thought about [Sphinx](http://sphinx-doc.org/) with [reStructuredText](http://docutils.sourceforge.net/rst.html), but that needs a working LaTeX environment to produce PDFs.

So, reportlab it is. It's not pretty, but you could work to make it so. I'm using the simplest version I can think of that produces something readable.

# Parsing the Excel

There are a few good Excel parsers these days, but [pandas](http://pandas.pydata.org/) is nice and standard. It also understands XLS files in addition to XLSX, which is important since we don't want to make people convert to XLSX by hand in order to use a different parse.

In this case, we need to know the structure of the XLS documents. There should be two sheets. The first (called "RawData") contains columns like "Path" "CourseCode" etc. and then "Question_1", "Question_2" etc. The second (called "QuestionMapper") contains "Question 1" etc. in column A and the text of the question in column B.

The first row is a header in both cases.

I have no idea how fragile this structure is, so I'll explicitly refer to the sheets by name. That way, there's a decent chance this script will break if someone changes the underlying format.

In [12]:
import pandas as pd
import numpy as np
import os
xl_filename = 'test_eval.xlsx'
pdf_filename = os.path.splitext(xl_filename)[0] + '.pdf'
answers = pd.io.excel.read_excel(xl_filename,sheetname='RawData')
questionmap = pd.io.excel.read_excel(xl_filename,sheetname='QuestionMapper')

We want a per-student list of questions and answers. My first thought is to stick everything into a dictionary. We want to make sure to return the results in the correct order, so we could use an ordered dict. I think it's easier just to keep an ordered list of questions.

In [13]:
questions = questionmap["Question"].values

In [14]:
print questions

[u'What were the most positive features of this course?'
 u'What is your assessment of the design, materials and assignments in this course?'
 u'How could this course be improved next time it is offered?'
 u"How well were\xa0[InstructorName] 's objectives (stated or implied) fulfilled?"
 u"What were [InstructorName]'s strongest contributions to this course?"
 u"How could [InstructorName]'s teaching be improved?"
 u'What influence did\xa0[InstructorName] have on your interest in this subject?'
 u'In the space below, please provide a statement about the quality of your performance in this course.'
 u'Students are expected to sign these forms (by adding your name), and should know that unsigned forms are unlikely to be taken seriously by evaluating committees.']


About the below code:

When we iterate through the rows, `idx` is the number of the row, and `qd` comes to us as the "question dictionary" where row 1 is expected to name the columns, and we can then look up entries by name. For example, column A happens to be "Column" and column B is "Question", so asking for `qd['Question']` gets the thing in column B.

`qm` is then my "question map": it maps something like "Question 1" to "What were the most positive features of this course"


In [15]:
#qm will map the column names (Question_1) to question text
qm = {}
for (idx,qd) in questionmap.iterrows():
    qn = qd['Column'].replace(' ','_')
    qt = qd['Question']
    qm[qn] = qt

In [16]:
print qm

{u'Question_3': u'How could this course be improved next time it is offered?', u'Question_2': u'What is your assessment of the design, materials and assignments in this course?', u'Question_1': u'What were the most positive features of this course?', u'Question_7': u'What influence did\xa0[InstructorName] have on your interest in this subject?', u'Question_6': u"How could [InstructorName]'s teaching be improved?", u'Question_5': u"What were [InstructorName]'s strongest contributions to this course?", u'Question_4': u"How well were\xa0[InstructorName] 's objectives (stated or implied) fulfilled?", u'Question_9': u'Students are expected to sign these forms (by adding your name), and should know that unsigned forms are unlikely to be taken seriously by evaluating committees.', u'Question_8': u'In the space below, please provide a statement about the quality of your performance in this course.'}


Now let's grab the data that should be common to all rows

In [17]:
path = answers.Path[0]
course_code = answers.CourseCode[0]
course_title = answers.CourseTitle[0]
instructor_name = answers.InstructorName[0]
enrollments = answers.Enrollments[0]

And now let's slurp up the data per student.

In [18]:
a = {}
for (idx,student) in answers.iterrows():
    a[idx] = {}
    for colname in answers.columns:
        col_name = colname.replace(' ','_')
        if col_name in qm:
            print "Looking up",col_name
            a[idx][qm[col_name]] = student[colname]
        else:
            print "Could not find",colname

Could not find Path
Could not find CourseCode
Could not find CourseTitle
Could not find UniqueID
Could not find InstructorName
Could not find Enrollments
Looking up Question_1
Looking up Question_2
Looking up Question_3
Looking up Question_4
Looking up Question_5
Looking up Question_6
Looking up Question_7
Looking up Question_8
Looking up Question_9
Could not find Path
Could not find CourseCode
Could not find CourseTitle
Could not find UniqueID
Could not find InstructorName
Could not find Enrollments
Looking up Question_1
Looking up Question_2
Looking up Question_3
Looking up Question_4
Looking up Question_5
Looking up Question_6
Looking up Question_7
Looking up Question_8
Looking up Question_9
Could not find Path
Could not find CourseCode
Could not find CourseTitle
Could not find UniqueID
Could not find InstructorName
Could not find Enrollments
Looking up Question_1
Looking up Question_2
Looking up Question_3
Looking up Question_4
Looking up Question_5
Looking up Question_6
Looking up

Now we're ready to stamp out the text, believe it or not. The only cute thing is that `pandas` uses nan ("not a number") to represent missing data. We'll use `numpy` (imported above as `np`) to test for nan, and turn it into "No answer given."

In [19]:
a[0][questions[0]]

u'The materials we went over was interesting'

In [20]:
def is_nan(x): 
    try: return np.isnan(x) 
    except: return False #isnan only eats strings

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet # http://eric.sau.pe/tag/reportlab-getsamplestylesheet/
from reportlab.rl_config import defaultPageSize
from reportlab.lib.units import inch

PAGE_HEIGHT=defaultPageSize[1]
PAGE_WIDTH=defaultPageSize[0]
styles = getSampleStyleSheet()

pageinfo = course_code

def myFirstPage(canvas, doc):
    canvas.saveState()
    canvas.setFont('Times-Bold',16)
    canvas.drawCentredString(PAGE_WIDTH/2.0, PAGE_HEIGHT-48, course_title)
    canvas.drawCentredString(PAGE_WIDTH/2.0, PAGE_HEIGHT-68, course_code)
    canvas.drawCentredString(PAGE_WIDTH/2.0, PAGE_HEIGHT-88, instructor_name)
    canvas.drawCentredString(PAGE_WIDTH/2.0, PAGE_HEIGHT-108, "Answers from {a} of {b} enrolled students".format(a=len(a),
                                                                                                                b=enrollments))

    canvas.setFont('Times-Roman',9)
    canvas.drawString(inch, 0.75 * inch,"First Page / %s" % pageinfo)
    canvas.restoreState()
    
def myLaterPages(canvas, doc):
    canvas.saveState()
    canvas.setFont('Times-Roman', 9)
    canvas.drawString(inch, 0.75 * inch,"Page %d %s" % (doc.page, pageinfo))
    canvas.restoreState()
    
def go():
    doc = SimpleDocTemplate(pdf_filename)
    Story = [Spacer(1,2*inch)]
    style = styles["Normal"]
    for idx in sorted(a):
        Story.append(Paragraph("Student {i} ({n})".format(i=idx+1, n=a[idx][questions[-1]]),styles['Heading1']))
        for question in questions[:-1]:
            #Story.append(Paragraph(question,styles['Heading3']))
            answer = a[idx][question]
            question_cor_name = question.replace("[InstructorName]", instructor_name)
            if u'\xa0' in question_cor_name: question_cor_name = question_cor_name.replace(u'\xa0', u' ') #Corrects for unicode encoding error
            if is_nan(answer):
                answer = 'No answer given.'
            #Story.append(Paragraph(answer,styles['Normal']))
            Story.append(Paragraph("<b> {q} </b><br/>{a}".format(q=question_cor_name,a=answer),styles['Normal']))
        Story.append(Paragraph("<b> </b><br/>",styles['Normal'])) # Hack for spacing
    #for i in range(100):
    #    bogustext = ("Paragraph number %s. " % i) *20
    #    p = Paragraph(bogustext, style)
    #    Story.append(p)
    #    Story.append(Spacer(1,0.2*inch))
    doc.build(Story, onFirstPage=myFirstPage, onLaterPages=myLaterPages)

In [21]:
go()